Research Paper to look at for multi label classification:

"An extensive experimental comparison of methods for multi-label learning" 
https://kt.ijs.si/DragiKocev/wikipage/lib/exe/fetch.php?media=2012pr_ml_comparison.pdf

### Approaching Multi Label Problem:
1. [Detailed Article (Analytics Vidya)](https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/)
2. [Model Evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html)
3. [sklearn multi label model](https://scikit-learn.org/stable/modules/multiclass.html#multilabel-classification-format)
4. [For more details visit sklearn-multi-label](http://scikit.ml/api/skmultilearn.html#module-skmultilearn.ensemble)
   - Approaches:
       - Problem Transformation :
           * Three different ways:
               - Binary Relevance : treats each label as a separate single class classification problem. 
               - Classifier Chains : first classifier is trained just on the input data and then each next classifier is trained on the input space and all the previous classifiers in the chain.  
               - Label Powerset : transform the problem into a multi-class problem with one multi-class classifier is trained on all unique label combinations found in the training data. 
       - Adapted Algorithm : adapting the algorithm to directly perform multi-label classification, rather than transforming the problem into different subsets of problems. 
       - Ensemble approaches : ensembling classification functions, which we can use for obtaining better results. 

In [25]:
!pip install iterative-stratification

In [ ]:
!pip install scikit-multilearn

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
# from sklearn.compose import make_column_transformer
# from sklearn.pipeline import make_pipeline
# import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
#from textblob import TextBlob
from bs4 import BeautifulSoup
import unicodedata
# import gensim
import re
import en_core_web_sm
nlp = en_core_web_sm.load()
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [6]:
# import ast
# data['Tags'] = [ast.literal_eval(item) for item in data['Tags']]

In [3]:
df= pd.read_json('cleaned_stackoverflow.json',lines= True, orient = 'records')

In [4]:
df = df[df['filtered_tags'].map(lambda x:  len(x)) >0]
df.reset_index(drop=True,inplace=True)

In [5]:
# df = df.sample(frac = 0.5).reset_index(drop = True)

In [6]:
df.shape

(200742, 3)

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['filtered_tags'])

In [9]:
# import joblib
# joblib.dump(multilabel,'multilabel_binarizer_3.pkl.gz')

In [10]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.multiclass import OneVsRestClassifier
# from zeugma.embeddings import EmbeddingTransformer


from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
# glove = EmbeddingTransformer('glove')
tfidf = TfidfVectorizer(stop_words = stop_words,max_features= 100000, ngram_range = (1,4))
X = tfidf.fit_transform(df['cleaned_text'])
# X = glove.transform(df['cleaned_text'])



In [11]:
# import pickle
# pickle.dump(tfidf, open("tfidf_file_100k.pkl.gz", "wb"))

In [12]:
# handle imbalance data
%%time
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
mskf = MultilabelStratifiedShuffleSplit(n_splits=10, random_state=0, test_size = 0.33)

for train_index, test_index in mskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [     0      2      5 ... 200738 200739 200740] TEST: [     1      3      4 ... 200728 200733 200741]
TRAIN: [     0      3      4 ... 200737 200739 200740] TEST: [     1      2      5 ... 200731 200738 200741]
TRAIN: [     0      1      3 ... 200737 200738 200740] TEST: [     2      6     11 ... 200735 200739 200741]
TRAIN: [     0      1      3 ... 200739 200740 200741] TEST: [     2      4      7 ... 200732 200737 200738]
TRAIN: [     0      1      4 ... 200737 200739 200740] TEST: [     2      3      6 ... 200734 200738 200741]
TRAIN: [     0      1      2 ... 200738 200739 200741] TEST: [     3      9     10 ... 200736 200737 200740]
TRAIN: [     1      2      4 ... 200738 200739 200740] TEST: [     0      3      8 ... 200735 200737 200741]
TRAIN: [     0      1      2 ... 200738 200740 200741] TEST: [    16     17     19 ... 200732 200733 200739]
TRAIN: [     1      2      3 ... 200738 200739 200741] TEST: [     0      9     11 ... 200729 200731 200740]
TRAIN: [     0     

In [23]:
# multilabel.classes_

In [13]:
y_test.shape , X_test.shape

((66345, 343), (66345, 100000))

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
# from skmultilearn.problem_transform import BinaryRelevance
# from skmultilearn.problem_transform import LabelPowerset
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import ClassifierChain



from sklearn.metrics import jaccard_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score


In [15]:


# neigh = KNeighborsClassifier(n_neighbors=10, weights = 'distance', n_jobs = -1)

# svm = SVC(class_weight = 'balanced', kernel = 'poly', C= 1.5)
lr = LogisticRegression(n_jobs=-1,class_weight = 'balanced',penalty = 'l2',solver = 'liblinear',C = 10,max_iter = 90,verbose =1)
# lrcv = LogisticRegressionCV(class_weight = 'balanced',n_jobs = -1)
# rfc = RandomForestClassifier( random_state =0, class_weight = 'balanced',n_jobs=-2,verbose = 1)

# xgbc = xgb.XGBClassifier(objective = 'binary:logistic',n_jobs=-1,eval_metric = 'logloss')

In [16]:
import multiprocessing
multiprocessing.cpu_count()

8

In [15]:
# parameters1 = {'base_estimator__penalty': ['l1','l2'],
#  'base_estimator__C': [1,5,10],
#  'base_estimator__max_iter': [100,200,300],
#  'base_estimator__solver': ['saga','liblinear'],
#  'base_estimator__class_weight':['balanced']}

In [16]:
# from sklearn.model_selection import RandomizedSearchCV
# lr = LogisticRegression(n_jobs=-1)
# randomclf = ClassifierChain(lr)
# grid_random = RandomizedSearchCV(randomclf,parameters1,cv=3, scoring = 'f1_micro',verbose = 2, n_jobs=-1)


In [17]:
# %time
# grid_random.fit(X_train,y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 273.9min finished


RandomizedSearchCV(cv=3,
                   estimator=ClassifierChain(base_estimator=LogisticRegression(n_jobs=-1)),
                   n_jobs=-1,
                   param_distributions={'base_estimator__C': [1, 5, 10],
                                        'base_estimator__class_weight': ['balanced'],
                                        'base_estimator__max_iter': [100, 200,
                                                                     300],
                                        'base_estimator__penalty': ['l1', 'l2'],
                                        'base_estimator__solver': ['saga',
                                                                   'liblinear']},
                   scoring='f1_micro', verbose=2)

In [18]:
# grid_random.best_params_

{'base_estimator__solver': 'liblinear',
 'base_estimator__penalty': 'l2',
 'base_estimator__max_iter': 100,
 'base_estimator__class_weight': 'balanced',
 'base_estimator__C': 10}

In [21]:
# grid_random.cv_results_

{'mean_fit_time': array([16364.99956846,   238.53623827,   331.99592233,   273.62559605,
         2085.75467992,   173.08717108,  1029.68074536,   175.19045432,
          203.2115198 ,  5778.14145684]),
 'std_fit_time': array([54.68185367,  1.17771675, 25.44385829,  4.8947151 ,  7.88262688,
         1.10946569, 13.3857735 ,  1.0809735 ,  2.75541256, 29.57573693]),
 'mean_score_time': array([18.88681467, 24.10120869, 24.92224828, 24.43849063, 28.73121031,
        25.14264297, 31.12573322, 25.07392907, 25.00705226, 20.43749984]),
 'std_score_time': array([0.20855723, 0.67549271, 0.94913373, 0.72973119, 1.03354031,
        0.12577041, 2.28074402, 0.32035158, 0.52347147, 0.06755386]),
 'param_base_estimator__solver': masked_array(data=['saga', 'liblinear', 'liblinear', 'liblinear', 'saga',
                    'liblinear', 'saga', 'liblinear', 'liblinear', 'saga'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_val

In [22]:
# pd.DataFrame(grid_random.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_estimator__solver,param_base_estimator__penalty,param_base_estimator__max_iter,param_base_estimator__class_weight,param_base_estimator__C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,16364.999568,54.681854,18.886815,0.208557,saga,l1,300,balanced,5,"{'base_estimator__solver': 'saga', 'base_estimator__penalty': 'l1', 'base_estimator__max_iter': 300, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 5}",0.494220,0.492339,0.495475,0.494011,0.001289,4
1,238.536238,1.177717,24.101209,0.675493,liblinear,l2,100,balanced,10,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l2', 'base_estimator__max_iter': 100, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 10}",0.539959,0.535831,0.534864,0.536885,0.002210,1
2,331.995922,25.443858,24.922248,0.949134,liblinear,l1,200,balanced,10,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l1', 'base_estimator__max_iter': 200, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 10}",0.523182,0.521790,0.516889,0.520620,0.002699,2
3,273.625596,4.894715,24.438491,0.729731,liblinear,l1,100,balanced,5,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l1', 'base_estimator__max_iter': 100, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 5}",0.522686,0.521231,0.515157,0.519691,0.003261,3
4,2085.754680,7.882627,28.731210,1.033540,saga,l2,200,balanced,5,"{'base_estimator__solver': 'saga', 'base_estimator__penalty': 'l2', 'base_estimator__max_iter': 200, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 5}",0.102680,0.087538,0.097530,0.095916,0.006286,9
5,173.087171,1.109466,25.142643,0.125770,liblinear,l2,300,balanced,1,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l2', 'base_estimator__max_iter': 300, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 1}",0.425543,0.430603,0.425059,0.427068,0.002507,6
6,1029.680745,13.385773,31.125733,2.280744,saga,l2,100,balanced,1,"{'base_estimator__solver': 'saga', 'base_estimator__penalty': 'l2', 'base_estimator__max_iter': 100, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 1}",0.035665,0.035911,0.034912,0.035496,0.000425,10
7,175.190454,1.080973,25.073929,0.320352,liblinear,l2,200,balanced,1,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l2', 'base_estimator__max_iter': 200, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 1}",0.425543,0.430603,0.425059,0.427068,0.002507,6
8,203.211520,2.755413,25.007052,0.523471,liblinear,l1,300,balanced,1,"{'base_estimator__solver': 'liblinear', 'base_estimator__penalty': 'l1', 'base_estimator__max_iter': 300, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 1}",0.447789,0.449802,0.444222,0.447271,0.002307,5
9,5778.141457,29.575737,20.437500,0.067554,saga,l1,200,balanced,1,"{'base_estimator__solver': 'saga', 'base_estimator__penalty': 'l1', 'base_estimator__max_iter': 200, 'base_estimator__class_weight': 'balanced', 'base_estimator__C': 1}",0.256804,0.266864,0.278125,0.267264,0.008709,8


In [17]:
%%time
for classifier in [lr]:
    clf3 = ClassifierChain(classifier, cv = 5)
    clf3.fit(X_train , y_train)


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear

In [14]:
# %%time
# for classifier in [rfc]:
#     clf3 = ClassifierChain(classifier, cv = 5)
#     clf3.fit(X_train , y_train)


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    1.9s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    2.9s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  87 out of 100 | elapsed:    2.1s remaining:    0.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent worke

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.5s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.6s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-2)]:

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    4.9s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    4.5s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    4.1s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done  87 out of 100 | elapsed:    2.1s remaining:    0.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.

[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   10.5s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    9.8s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   10.1s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_j

...

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.8s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.8s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-2)]:

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.2s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.1s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.5s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.5s
[Parallel(n_jobs=

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.9s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    3.9s finished
[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    0.0s
[Parallel(n_jobs=7)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-2)]:

CPU times: user 3h 17min 23s, sys: 1min 46s, total: 3h 19min 10s
Wall time: 2h 32min 21s


In [ ]:
# %%time
# for classifier in [svm]:
#     clf3 = ClassifierChain(classifier, cv = 5)
#     clf3.fit(X_train , y_train)

In [18]:
y_pred = clf3.predict(X_test)

In [34]:
x_pred =clf3.predict(X_train)

In [15]:
y_pred.shape

(66345, 343)

In [19]:
def score(y_true,y_pred):
    print(f"Jaccard_score: {jaccard_score(y_true, y_pred, average='micro')*100}")
    print(f"Hamming_loss: {hamming_loss(y_true, y_pred)}")
    print(f"Acuracy_score: {accuracy_score(y_true, y_pred)}")
    print('-------')

In [17]:
# RandomForest without hyperparameter tuning
score(y_test,y_pred)

Jaccard_score: 19.685615848406545
Hamming_loss: 0.004933780616213299
Acuracy_score: 0.1088626739261948
-------


In [20]:
# Logistic Regression with hyperparameter tuning
score(y_test,y_pred)

Jaccard_score: 41.52203182655767
Hamming_loss: 0.005592201028856361
Acuracy_score: 0.1552942949732459
-------


In [41]:
clf3

ClassifierChain(base_estimator=LogisticRegression(class_weight='balanced',
                                                  n_jobs=-1))

In [18]:
# Random Forest
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average = 'micro')

0.32895541722140265

In [23]:
# Logistic Regression with Hyperparameter tuning
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average = 'weighted')

0.6005683756945229

In [24]:
# Logistic Regression with Hyperparameter tuning
from sklearn.metrics import precision_score
precision_score(y_test, y_pred, average = 'micro')

0.521326986874369

In [25]:
# Logistic Regression with Hyperparameter tuning
from sklearn.metrics import recall_score
recall_score(y_test,y_pred,average = 'micro')

0.6710607422150596

In [21]:
# Logistic Regression with Hyperparameter tuning
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.70      0.77      0.73       130
           1       0.23      0.31      0.27       602
           2       0.34      0.52      0.41       405
           3       0.57      0.64      0.61       413
           4       0.32      0.44      0.37       311
           5       0.54      0.66      0.59       140
           6       0.72      0.81      0.76       230
           7       0.67      0.80      0.73       748
           8       0.50      0.69      0.58       105
           9       0.73      0.82      0.77      3463
          10       0.40      0.70      0.51       117
          11       0.15      0.32      0.21       118
          12       0.74      0.89      0.81       149
          13       0.27      0.47      0.34       434
          14       0.77      0.81      0.79      1885
          15       0.52      0.75      0.61       158
          16       0.15      0.34      0.21       184
          17       0.76    

### Testing of model on new Stackoverflow Question from website

In [21]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

def data_clean(text):
    text = text.lower()  # lowering of characters
    text = text.strip()  # removeing extra spaces before and after the sentence
    # text = TextBlob(text).correct() # spelling mistakes correction  (very slow) (maybe try directly on dataframe column)
    return text


# remove accented characters (Sómě Áccěntěd těxt)
def remove_accented_chars(text):
    text = (
        unicodedata.normalize("NFKD", text)
        .encode("ascii", "ignore")
        .decode("utf-8", "ignore")
    )
    return text


def strip_html_tags(text):  # function for removing html tags
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text


def remove_special_characters(text, remove_digits=False):  # function for removing punctuations , before using this remove any contractions(like "I'll" --> "I will") in text data
    pattern = r"[^a-zA-z0-9\s\-\+\#]" if not remove_digits else r"[^a-zA-z\s\-\+\#]"
    text = re.sub(pattern, "", text)
    return text

# #another way to remove punctuation
# import string
# def remove_punctuation(text):
#     pattern = string.punctuation.replace('+','').replace('-','')
#     text_p = text.translate(str.maketrans('', '', pattern))
#     return text_p

# custom way to remove punctuations of your choice from all of them (than regex one)
# puncts = [',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '*', '\\', '•',  '~', '@', '£',
#  '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
#  '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
#  '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
#  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

# def punctuation(text):  # before using this remove any contractions(like "I'll" --> "I will") in text data
#     x = str(text)
#     for punct in puncts:
#         if punct in x :
#             x = x.replace(punct, '')
#     return x


def remove_links_emojis(text):
    pattern = re.compile( "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")  # removing https:www.examples.com
    text = pattern.sub("", text)

    emoji = re.compile("["u"\U0001F600-\U0001FFFF"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251""]+",
        flags=re.UNICODE,
    )
    text = emoji.sub(r"", text)
    return text

def contractions(text):  # contraction of text i.e remove_apostrophes
    text = re.sub("ain't", "is not", str(text))
    text = re.sub("aren't", "are not", str(text))
    text = re.sub("can't", "cannot", str(text))
    text = re.sub("can't've", "cannot have", str(text))
    text = re.sub("'cause", "because", str(text))
    text = re.sub("could've", "could have", str(text))
    text = re.sub("couldn't", "could not", str(text))
    text = re.sub("couldn't've", "could not have", str(text))
    text = re.sub("didn't", "did not", str(text))
    text = re.sub("doesn't", "does not", str(text))
    text = re.sub("don't", "do not", str(text))
    text = re.sub("hadn't", "had not", str(text))
    text = re.sub("hadn't've", "had not have", str(text))
    text = re.sub("hasn't", "has not", str(text))
    text = re.sub("haven't", "have not", str(text))
    text = re.sub("he'd", "he would", str(text))
    text = re.sub("he'd've", "he would have", str(text))
    text = re.sub("he'll", "he will", str(text))
    text = re.sub("he'll've", "he he will have", str(text))
    text = re.sub("he's", "he is", str(text))
    text = re.sub("how'd", "how did", str(text))
    text = re.sub("how'd'y", "how do you", str(text))
    text = re.sub("how'll", "how will", str(text))
    text = re.sub("how's", "how is", str(text))
    text = re.sub("I'd", "I would", str(text))
    text = re.sub("I'd've", "I would have", str(text))
    text = re.sub("I'll", "I will", str(text))
    text = re.sub("I'll've", "I will have", str(text))
    text = re.sub("I'm", "I am", str(text))
    text = re.sub("I've", "I have", str(text))
    text = re.sub("i'd", "i would", str(text))
    text = re.sub("i'd've", "i would have", str(text))
    text = re.sub("i'll", "i will", str(text))
    text = re.sub("i'll've", "i will have", str(text))
    text = re.sub("i'm", "i am", str(text))
    text = re.sub("i've", "i have", str(text))
    text = re.sub("isn't", "is not", str(text))
    text = re.sub("it'd", "it would", str(text))
    text = re.sub("it'd've", "it would have", str(text))
    text = re.sub("it'll", "it will", str(text))
    text = re.sub("it'll've", "it will have", str(text))
    text = re.sub("it's", "it is", str(text))
    text = re.sub("let's", "let us", str(text))
    text = re.sub("ma'am", "madam", str(text))
    text = re.sub("mayn't", "may not", str(text))
    text = re.sub("might've", "might have", str(text))
    text = re.sub("mightn't", "might not", str(text))
    text = re.sub("mightn't've", "might not have", str(text))
    text = re.sub("must've", "must have", str(text))
    text = re.sub("mustn't", "must not", str(text))
    text = re.sub("mustn't've", "must not have", str(text))
    text = re.sub("needn't", "need not", str(text))
    text = re.sub("needn't've", "need not have", str(text))
    text = re.sub("o'clock", "of the clock", str(text))
    text = re.sub("oughtn't", "ought not", str(text))
    text = re.sub("oughtn't've", "ought not have", str(text))
    text = re.sub("shan't", "shall not", str(text))
    text = re.sub("sha'n't", "shall not", str(text))
    text = re.sub("shan't've", "shall not have", str(text))
    text = re.sub("she'd", "she would", str(text))
    text = re.sub("she'd've", "she would have", str(text))
    text = re.sub("she'll", "she will", str(text))
    text = re.sub("she'll've", "she will have", str(text))
    text = re.sub("she's", "she is", str(text))
    text = re.sub("should've", "should have", str(text))
    text = re.sub("shouldn't", "should not", str(text))
    text = re.sub("shouldn't've", "should not have", str(text))
    text = re.sub("so've", "so have", str(text))
    text = re.sub("so's", "so as", str(text))
    text = re.sub("that'd", "that would", str(text))
    text = re.sub("that'd've", "that would have", str(text))
    text = re.sub("that's", "that is", str(text))
    text = re.sub("there'd", "there would", str(text))
    text = re.sub("there'd've", "there would have", str(text))
    text = re.sub("there's", "there is", str(text))
    text = re.sub("they'd", "they would", str(text))
    text = re.sub("they'd've", "they would have", str(text))
    text = re.sub("they'll", "they will", str(text))
    text = re.sub("they'll've", "they will have", str(text))
    text = re.sub("they're", "they are", str(text))
    text = re.sub("they've", "they have", str(text))
    text = re.sub("to've", "to have", str(text))
    text = re.sub("wasn't", "was not", str(text))
    text = re.sub("we'd", "we would", str(text))
    text = re.sub("we'd've", "we would have", str(text))
    text = re.sub("we'll", "we will", str(text))
    text = re.sub("we'll've", "we will have", str(text))
    text = re.sub("we're", "we are", str(text))
    text = re.sub("we've", "we have", str(text))
    text = re.sub("weren't", "were not", str(text))
    text = re.sub("what'll", "what will", str(text))
    text = re.sub("what'll've", "what will have", str(text))
    text = re.sub("what're", "what are", str(text))
    text = re.sub("what's", "what is", str(text))
    text = re.sub("what've", "what have", str(text))
    text = re.sub("when's", "when is", str(text))
    text = re.sub("when've", "when have", str(text))
    text = re.sub("where'd", "where did", str(text))
    text = re.sub("where's", "where is", str(text))
    text = re.sub("where've", "where have", str(text))
    text = re.sub("who'll", "who will", str(text))
    text = re.sub("who'll've", "who will have", str(text))
    text = re.sub("who's", "who is", str(text))
    text = re.sub("who've", "who have", str(text))
    text = re.sub("why's", "why is", str(text))
    text = re.sub("why've", "why have", str(text))
    text = re.sub("will've", "will have", str(text))
    text = re.sub("won't", "will not", str(text))
    text = re.sub("won't've", "will not have", str(text))
    text = re.sub("would've", "would have", str(text))
    text = re.sub("wouldn't", "would not", str(text))
    text = re.sub("wouldn't've", "would not have", str(text))
    text = re.sub("y'all", "you all", str(text))
    text = re.sub("y'all'd", "you all would", str(text))
    text = re.sub("y'all'd've", "you all would have", str(text))
    text = re.sub("y'all're", "you all are", str(text))
    text = re.sub("y'all've", "you all have", str(text))
    text = re.sub("you'd", "you would", str(text))
    text = re.sub("you'd've", "you would have", str(text))
    text = re.sub("you'll", "you will", str(text))
    text = re.sub("you'll've", "you will have", str(text))
    text = re.sub("you're", "you are", str(text))
    text = re.sub("you've", "you have", str(text))
    return text


# document = document.replace(‘…’, ‘’) # small nuances
# document = document.replace(‘Mr.’, ‘Mr’).replace(‘Mrs.’, ‘Mrs’) # small nuances to look for


def remove_hashtags(text):
    for item in text.split():
        if item.startswith("#"):
            text = text.append(item)
    return text


def lemmatize_text(text):  # function for lemmetization of text
    text = nlp(text)
    text = str( " ".join([word.lemma_ if word.lemma_ != "-PRON-" else word.text for word in text]))
    return text

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [22]:
def preprocess(text, clean_data = True , 
               contraction_expansion = True,
               accented_char_removal = True,
               text_lemmatization = True, link_emoji = True,
               strip_html_tag = True, 
               special_characters = True,
               hashtags = True,remove_digits = False,
               stop_words = True):
    
    # cleaned data (lower & strip whitespaces & spelling mistake)
    if clean_data:
        text = data_clean(text)
    #strip html
    if strip_html_tag:
        text = strip_html_tags(text)
    # accented char removal
    if accented_char_removal:
        text = remove_accented_chars(text)
    # hashtags
    if hashtags:
        text = remove_hashtags(text)
    # exapand contraction
    if contraction_expansion:
        text = contractions(text)
    if stop_words:
        text = remove_stopwords(text)
    # lemmetization
    if text_lemmatization:
        text = lemmatize_text(text)
    # punctuations and digits
    if special_characters:
        text = remove_special_characters(text,remove_digits = remove_digits)

    # remove extra whitespace
    text = re.sub(' +', ' ', text)

    
    
    return text

In [92]:
x = """


I am working on an interactive visualization of the world happiness report from the years 2015 up to 2020. The data was split into 6 csv files. Using pandas, I have succesfully cleaned the data and concatenated them into one big JSON file with the following format:

[
  {
    "Country": "Switzerland",
    "Year": 2015,
    "Happiness Rank": 1,
    "Happiness Score": 7.587000000000001,
  },
  {
    "Country": "Iceland",
    "Year": 2015,
    "Happiness Rank": 2,
    "Happiness Score": 7.561,
  },
  {
    "Country": "Switzerland",
    "Year": 2016,
    "Happiness Rank": 2,
    "Happiness Score": 7.5089999999999995,
  },
  {
    "Country": "Iceland",
    "Year": 2016,
    "Happiness Rank": 3,
    "Happiness Score": 7.501,
  },
  {
    "Country": "Switzerland",
    "Year": 2017,
    "Happiness Rank": 3,
    "Happiness Score": 7.49399995803833,
  },
  {
    "Country": "Iceland",
    "Year": 2017,
    "Happiness Rank": 1,
    "Happiness Score": 7.801,
  }
]

Now, I would like to programmatically format the JSON file such that it has the following format:

{
    "2015": {
        "Switzerland": {
            "Happiness Rank": 1,
            "Happiness Score": 7.587000000000001
        },
        "Iceland": {
            "Happiness Rank": 2,
            "Happiness Score": 7.561
        }
    },
    "2016": {
        "Switzerland": {
            "Happiness Rank": 2,
            "Happiness Score": 7.5089999999999995
        },
        "Iceland": {
            "Happiness Rank": 3,
            "Happiness Score": 7.501
        }
    },
    "2017": {
        "Switzerland": {
            "Happiness Rank": 3,
            "Happiness Score": 7.49399995803833
        },
        "Iceland": {
            "Happiness Rank": 1,
            "Happiness Score": 7.801
        }
    }
}

It has to be done programmatically, since there are over 900 distinct (country, year) pairs. I want the JSON in this format since it make the JSON file more readable, and makes it easier to select appropriate data. If I want the rank of Iceland in 2015, I can then do data[2015]["Iceland"]["Happiness Rank"]

Does anyone know the easiest / most convenient way to do this in Python?

"""

### Loading Saved Logistic Regression model

In [108]:
import joblib
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
clf2 = joblib.load("logreg100_multi_label2.pkl.gz")
multilabel = joblib.load("multilabel_binarizer_3.pkl.gz")

tfidf_vocab = pickle.load(open("tfidf_file_100k.pkl.gz", "rb"))
tfidf = TfidfVectorizer(
    max_features=100000,
    ngram_range=(1, 4),
    stop_words="english",
    vocabulary=tfidf_vocab.vocabulary_,
)

In [33]:
tfidf_vocab.vocabulary_

{'deal': 11623,
 'role': 36263,
 'cross': 10224,
 'project': 33049,
 'one': 29753,
 'important': 21264,
 'question': 34174,
 'face': 15592,
 'gcp': 18497,
 'data': 10653,
 'engineer': 14071,
 'practice': 32376,
 'exam': 14964,
 'organization': 30347,
 'expand': 15302,
 'usage': 45005,
 'many': 25985,
 'team': 41977,
 'start': 40138,
 'create': 9793,
 'project one': 33153,
 'start create': 40152,
 'webview': 48432,
 'work': 48940,
 'plain': 31778,
 'setcontentview': 38472,
 'want': 47770,
 'use': 45012,
 'show': 38683,
 'html': 20430,
 'code': 7437,
 'photo': 31525,
 'think': 42474,
 'simple': 38963,
 'way': 48087,
 'problem': 32673,
 'file': 16049,
 'app': 2645,
 'copy': 9450,
 'new': 28531,
 'know': 23571,
 'even': 14772,
 'want use': 48038,
 'use webview': 46198,
 'show html': 38750,
 'html code': 20442,
 'simple way': 39018,
 'problem file': 32725,
 'file app': 16071,
 'copy new': 9467,
 'new project': 28656,
 'project know': 33130,
 'group': 19716,
 'arbitrary': 3241,
 'datum': 112

In [109]:
import time
t = time.time()
px = preprocess(x, hashtags =False, remove_digits =True)
xt = tfidf.fit_transform([px])
predi = clf2.predict_proba(xt)
print(f"Preprocessing takes {time.time()-t} secs")

Preprocessing takes 0.4368298053741455 secs


In [110]:
# import time
# t = time.time()
# px = preprocess(x, hashtags =False, remove_digits =True)
# xt = tfidf.transform([px])
# predi = clf3.predict_proba(xt)
# print(f"Preprocessing takes {time.time()-t} secs")

Preprocessing takes 0.35710930824279785 secs


In [87]:
px

'use matplotlib library pyhton would like plot graphs dynamic variable ie variable would change accord another variable state plot function import data frame extract different gas concentration m_conc flux m_fluxes m_conc ec_top [ ppbv ] m_conc ec_top [ ncps ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_conc ec_top [ ppbv ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] m_flux ec_top [ flux_m ] want able plot evolution gas concentration fluxes time one function would allow choose plot concentration flux gas write far color_ black graph_type conc fig ax ax ax ax ax ax ax ax ax pltsubplots sharex true sharey false figsuptitle influence wind direction bvocs concentration axp

In [88]:
def get_tags(predicted_list):
    mlb =[(i1,c1)for i1, c1 in enumerate(multilabel.classes_)]
    temp_list = sorted([(i,c) for i,c in enumerate(list(predicted_list))],key = lambda x: x[1], reverse=True)
    tag_list = [item1 for item1 in temp_list if item1[1]>=0.5]
    tags = [item[1] for item2 in tag_list[:5] for item in mlb if item2[0] == item[0] ]
    return tags

In [111]:
get_tags(predi[0])

['json', 'sorting', 'python', '.net']

In [60]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred1, average='micro')

0.8494070052399937

In [ ]:
# def infer_tags(text):
#     text = preprocess(text)
#     text_vec = tfidf_vectorizer.transform([text])
#     text_pred = clf.predict(text_vec)
#     return multilabel_binarizer.inverse_transform(text_pred)